In [1]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 280 kB 50.2 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Import Necessary Libraries

In [3]:
import dgl
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from dgl.data import DGLDataset
from collections import OrderedDict
from dgl.nn import GraphConv
import networkx as nx
import matplotlib.pyplot as plt


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


# Process Dataset

In [4]:

############# Method  for getting labels ( within label dictionary) ##############
def get_label_dict(filename): 
  label_dict = OrderedDict()
  with open(filename, 'r') as labelfile: 
    for line in labelfile: 
      line = line.split(' ')
      line[1] = int(line[1].rstrip('\n'))

      label_dict[int(line[0])] = line[1]
  num_of_classes = len(set(label_dict.values()))
  return label_dict, num_of_classes

###################################################################################

###################### Process the dataset from files ###########################
class ProcessDataset(DGLDataset):
    def __init__(self, graph, label_file):
        
        # taking the initial graph as input graph ################
        self.inp_graph = graph
        self.label_file = label_file
        self.graph = None
        # self.edge_list = edge_list
        # self.label_dict = label_dict
        # print(self.label_dict)

    def process(self):
  
        G = self.inp_graph

        # making the graph bi-directional 
        edge_list = list()
        for u, v in G.edges():
          edge_list.append((u,v))
          edge_list.append((v,u))
        
        # getting labels and number of classes 
        label_dict, num_of_classes = get_label_dict(self.label_file)

        min_node = min(list(label_dict.keys()))
        new_label_dict = dict()

        min_target = min(list(set(label_dict.values())))

        if min_node == 1 and min_target == 1: 
          for key, value in label_dict.items():
            new_label_dict[key-1] = value -1 

          nodes_data = list(new_label_dict.keys())

          for i in range (len(edge_list)):
            u, v = edge_list[i]
            u = u - 1
            v = v - 1
            edge_list[i] = (u, v)   

        else:
          nodes_data = list(label_dict.keys())

        edges_data = edge_list

        degree = torch.tensor([1 for node in G.nodes()])
        node_features = torch.diag(degree)

        
        node_labels = torch.tensor([int(value) for value in label_dict.values()])
        if min_target == 1:
          node_labels = torch.tensor([int(value) for value in new_label_dict.values()])

        edges_src = torch.tensor([u for u, v in edges_data])
        edges_dst = torch.tensor([v for u, v in edges_data])
        #edges_dst = torch.from_numpy(edges_data['Dst'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=len(nodes_data))
        self.graph = dgl.add_self_loop(self.graph)
        self.graph.ndata['feat'] = node_features
        self.graph.ndata['label'] = node_labels


        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = len(nodes_data)
        n_train = int(n_nodes * 0.4)
        n_val = int(n_nodes * 0.2)

        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)

        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

        return self.graph, num_of_classes

    def get_graph(self):
      return self.graph
    # def __getitem__(self, i):
    #     return self.graph

    def __len__(self):
        return 1

#####################################################################################

# Graph Decomposition 

In [27]:

# Getting max truss or core value 
# Here tp means decomposition type 'truss' or 'core' or 'shell' 

################################### getting maximum core / truss value ##############################
def get_max_k_tp(G, tp): 
  i = 1
  if tp == 'truss':
    i = 2
    
  while(1):
    if tp == 'truss':
      Gk = nx.k_truss(G, i)
    elif tp == 'core':
      Gk = nx.k_core(G, i)
    elif tp == 'shell':
      Gk = nx.k_shell(G, i)
    #print(Gk, i)
    if len(Gk.edges()) == 0:
      return i-1, tp
    i += 1
  return i-1, tp

################################################################################


#################### Get the truss / core components, then extract chains and spanning tree, finally merge them ###################
# Although the function's name is truss compression, it has also compressed based on core or shell regarding the value of 'tp' 
def process_truss_or_core_components(G_k, max_val, tp , percentage_og_chains ,super_node_dictionary, sup_neighbor_dict):

  Gs = nx.Graph()

  remove_list = []
  graph_list = []

  min_val = 1

  for i in range(max_val, min_val, -1):
    #print(i)
    
    if tp == 'truss':
      G_t = nx.k_truss(G_k,i)
    elif tp == 'core':
      G_t = nx.k_core(G_k,i)
    elif tp == 'shell':
      G_t = nx.k_shell(G_k,i)

    # Get the connected components of k-truss/k-core subgraph
    cck = nx.connected_components(G_t)
    cck = list(cck)

    # Operations for reducing componests edges 
    for cc in cck: 

      subG = G_t.subgraph(cc)

      if len(cc) > 10:
        # Extract chains from the component 
        chains = sorted(list(nx.chain_decomposition(subG)), key=len)
        chains.reverse()
        
        ln = 0
        ln = int(len(chains)*percentage_og_chains)
        cns = chains[:ln]

        edges = []
        for item in cns: 
          edges.extend(item)
      
        eSubG = G_t.edge_subgraph(edges)
    
        # Get maximum spanning tree from the component
        T = nx.maximum_spanning_tree(subG)
        T = nx.compose(T,eSubG)
        T_m = T.copy()

        # save each component into a graph list
        graph_list.append(T_m)
      else:
        graph_list.append(subG)


    if i == max_val:
      Gt = nx.maximum_spanning_tree(G_k)
      Gs = nx.compose(Gs, Gt)
 
    sup_dict = dict()
    sup_dict_Ngb = dict()
    
  return Gs, super_node_dictionary, sup_neighbor_dict, graph_list

#############################################################

########## Compose the subgraphs from the graph list #####################
## gL is the graph list 
def graph_composition(G, gL):
  Gt = nx.Graph()
  sum = 0
  for item in gL: 
    Gt = nx.compose(Gt, item)
    #print(len(item), Gt)
  G_ = nx.compose(Gt,G)
  return G_


############### Whole sparification of k-core/ k-truss graph ################
def get_edge_reduced_composed_graph(G_k, tp , percentage_og_chains):
  Gmain = G_k.copy()

  ################ You can change 'core' / 'truss' 
  max_val, tp = get_max_k_tp(G_k, tp)

  global super_node_dictionary 
  global sup_neighbor_dict

  super_node_dictionary  = dict()
  sup_neighbor_dict = dict() 

  G, sup_node_dict, sup_node_nb_dict , gL= process_truss_or_core_components(G_k, max_val,tp,percentage_og_chains, super_node_dictionary, sup_neighbor_dict)
  compG = graph_composition(G, gL)

  return compG

################# Louvain decomposition #####################
from community import community_louvain

def louvain_composition(G, percentage_og_chains):
  T = nx.maximum_spanning_tree(G)
  components = community_louvain.best_partition(G)

  # getting Louvain communities
  group_dict = dict() 
  #print(components)
  for node, group in components.items(): 
    if group not in group_dict:
      group_dict[group] = [node]
    else:
      group_dict[group].append(node)
  Gp = nx.Graph()

  # Get subgraph for each community and extract chains and spanning tree and compose them 
  for key, value in group_dict.items():
    group_dict[key] = set(value)
    subG = G.subgraph(group_dict[key])
    subT = nx.maximum_spanning_tree(subG)

    chains = sorted(list(nx.chain_decomposition(subG)), key=len)
    chains.reverse()

    ln = 0
    ln = int(len(chains)*percentage_og_chains)
    cns = chains[:ln]

    edges = []
    for item in cns: 
      edges.extend(item)
 
    eSubG = G.edge_subgraph(edges)
    eSubG = nx.compose(eSubG, subT)

    Gp = nx.compose(Gp, eSubG)
    #Gp = nx.compose(Gp, T)
  return Gp

#############################################################################

# Set Up a GCN model

In [28]:

######### Setting up a GCN model for node embedding #############################

class GCN(nn.Module):
  def __init__(self, in_feats, h_feats, num_classes, num_of_layers):
    super(GCN, self).__init__()

    self.num_of_layers = num_of_layers
    self.conv_layers = nn.ModuleList()
    self.conv_layers.append(GraphConv(in_feats, h_feats))
 
    if num_of_layers > 2:
      for i in range(1, num_of_layers-1):
        self.conv_layers.append(GraphConv(h_feats, h_feats))
    self.conv_layers.append(GraphConv(h_feats, num_classes))
    #self.convN = GraphConv(h_feats, num_classes)

  def forward(self, g, in_feats, num_of_layers):

    h = self.conv_layers[0](g, in_feats)
    h = F.relu(h)

    if num_of_layers > 2: 
      for i in range(1, num_of_layers-1):
        h = self.conv_layers[i](g, h)
        h = F.relu(h)

    h = self.conv_layers[num_of_layers-1](g, h)

    return h


################# Train the Model ############################################
def train(model, g, num_of_layers):

  optimizer = torch.optim.Adam(model.parameters(), lr =0.005)
  best_val_acc = 0
  best_test_acc = 0
  
  features= g.ndata['feat']
  train_mask = g.ndata['train_mask']
  test_mask= g.ndata['test_mask']
  val_mask = g.ndata['val_mask']
  labels = g.ndata['label']
  
  #val_mask = g.ndata['val_mask']

  for e in range(251):

    #forward pass
    logits = model.forward(g, features, num_of_layers )
    
    #prediction
    pred = logits.argmax(1)
    
    #loss function
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    #acc 
    train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
    val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
    test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

    # Save the best validation accuracy and the corresponding test accuracy.
    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    #print(loss,train_acc)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e%100 == 0:
      print(f"In epoch {e}, loss: {loss}, training_acc: {train_acc}, test_acc: {test_acc}, best test acc: {best_test_acc}")

  return best_test_acc

# Experiment 

In [59]:
import networkx as nx

edgelist_file = None
label_file = None
dataset_no = int(input("Select Dataset -- 1 == cora; 2 == citeseer; 3 == pubmed \n"))
ds = ''
if dataset_no == 1: 
    ds ='Cora'
    
    edgelist_file = r'/content/drive/MyDrive/PhD Academic/Fall 22/Network Science/Project/Data/cora_edgelist_dgl.txt'
    label_file = r'/content/drive/MyDrive/PhD Academic/Fall 22/Network Science/Project/Data/cora_labels_dgl.txt'
elif dataset_no == 2:
    ds = 'CiteSeer'
    edgelist_file = r'/content/drive/MyDrive/PhD Academic/Fall 22/Network Science/Project/Data/citeseer_edgelist_dgl.txt'
    label_file = r'/content/drive/MyDrive/PhD Academic/Fall 22/Network Science/Project/Data/citeseer_labels_dgl.txt'
elif dataset_no == 3:
    ds = 'PubMed'
    edgelist_file = r'/content/drive/MyDrive/PhD Academic/Fall 22/Network Science/Project/Data/pubmed_edgelist_dgl.txt'
    label_file = r'/content/drive/MyDrive/PhD Academic/Fall 22/Network Science/Project/Data/pubmed_labels_dgl.txt'

G = nx.read_edgelist(edgelist_file, nodetype=int)
G.remove_edges_from(nx.selfloop_edges(G))
Gcopy = G.copy()

G_t_copy = G.copy()

print('k-Core/Truss')
tp = str(input('Enter the decomposion type \'core\' or \'truss\': '))
print('|Percentage of chains must be >= 0 and < 1|')
percentage_og_chains_truss = float(input('Percentage of chains Truss/Core: '))
while(percentage_og_chains_truss <= 0 or percentage_og_chains_truss > 1): 
  print('Percentage of chains must be >= 0 and < 1')
  percentage_og_chains_truss = float(input('Percentage of chains Truss/Core: '))

print('|Percentage of chains must be >= 0 and < 1|')
percentage_og_chains_louvain = float(input('Percentage of chains Louvain: '))
while(percentage_og_chains_louvain <= 0 or percentage_og_chains_louvain > 1):
  print('Percentage of chains must be >= 0 and < 1')
  percentage_og_chains_louvain = float(input('Percentage of chains Louvain: '))

print('|Number of Layers|')
num_of_layers = int(input('Number of layers: '))

##################### Node Classification Method ########################
def node_classification(G, label_file, num_of_layers):
  dataset = ProcessDataset(G, label_file)
  g1, num_of_classes = dataset.process()
  best_test_acc = 0

  model = GCN(g1.ndata['feat'].shape[1], 16, num_of_classes, num_of_layers )
  bset_test_acc = train(model, g1, num_of_layers)

  return bset_test_acc

#########################################################################

print(G,'\n')
composed_t_G = get_edge_reduced_composed_graph(Gcopy, tp, percentage_og_chains_truss)
print(composed_t_G,'\n')
composed_G = louvain_composition(G_t_copy, percentage_og_chains_louvain)
print(composed_G,'\n')
label_file = label_file

result_list = list()

for i in range(1):
  print('\n______Iter_______')
  print(i,'\n')

  best_test_acc_orgG = 0
  best_test_acc_compG = 0
  best_test_acc_t_compG = 0

  print('\nTraining_Original_Graph: ')
  best_test_acc_orgG = node_classification(G, label_file, num_of_layers)

  print('\nTraining_truss_Composed_Graph: ')
  best_test_acc_t_compG = node_classification(composed_t_G, label_file, num_of_layers)

  print('\nTraining_Louvain_Composed_Graph: ')
  best_test_acc_compG = node_classification(composed_G, label_file, num_of_layers)

  tup = (best_test_acc_orgG, best_test_acc_t_compG, best_test_acc_compG)
  result_list.append(tup)

#write the result to file 
u_sum = 0
v_sum = 0
w_sum = 0
for u, v, w in result_list:
  u_sum += u
  v_sum += v
  w_sum += w

org_avg = u_sum/len(result_list)
truss_avg = v_sum/ len(result_list)
louvain_avg = w_sum/ len(result_list)

with open('res_file.txt', 'a') as result_file: 
  result_file.write(str(num_of_layers))
  result_file.write('\t\t\t')
  result_file.write(str(round(org_avg.item(),4)))
  result_file.write('\t\t\t')
  result_file.write(str(percentage_og_chains_truss))
  result_file.write('\t\t\t')
  result_file.write(str(round(truss_avg.item(),4)))
  result_file.write('\t\t\t')
  result_file.write(str(percentage_og_chains_louvain))
  result_file.write('\t\t\t')
  result_file.write(str(round(louvain_avg.item(),4)))
  result_file.write('\n')
  result_file.close()

Select Dataset -- 1 == cora; 2 == citeseer; 3 == pubmed 
2
k-Core/Truss
Enter the decomposion type 'core' or 'truss': core
|Percentage of chains must be >= 0 and < 1|
Percentage of chains Truss/Core: 0.1
|Percentage of chains must be >= 0 and < 1|
Percentage of chains Louvain: 0.9
|Number of Layers|
Number of layers: 12
Graph with 3327 nodes and 4552 edges 

Graph with 3327 nodes and 4157 edges 

Graph with 3327 nodes and 4147 edges 


______Iter_______
0 


Training_Original_Graph: 
In epoch 0, loss: 1.7917225360870361, training_acc: 0.156390979886055, test_acc: 0.16516517102718353, best test acc: 0.16516517102718353
In epoch 100, loss: 0.935526430606842, training_acc: 0.6939849853515625, test_acc: 0.5135135054588318, best test acc: 0.554804801940918
In epoch 200, loss: 0.3943394720554352, training_acc: 0.8609022498130798, test_acc: 0.6081081032752991, best test acc: 0.6081081032752991

Training_truss_Composed_Graph: 
In epoch 0, loss: 1.7917758226394653, training_acc: 0.0714285746216

In [60]:
# best_test_acc_orgG, best_test_acc_t_compG, best_test_acc_compG
result_list

[(tensor(0.6081), tensor(0.6464), tensor(0.6434))]

In [50]:
org_avg , truss_avg, louvain_avg

(tensor(0.8127), tensor(0.8081), tensor(0.8044))